In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "qwiklabs-gcp-02-d4e1167507ba"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [4]:
model = GenerativeModel("gemini-1.5-flash")

In [5]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [6]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried flowers, playing on different themes:

**Evoking Timelessness & Enduring Beauty:**

* **Everlasting Blooms**
* **Dried & Divine**
* **The Timeless Bouquet**
* **Bloom Forever**
* **Eternally Blooming**
* **The Dried Flower Gallery**
* **Whispers of Bloom**

**Highlighting Unique & Vintage Appeal:**

* **The Vintage Bloom**
* **Botanicals & Co.**
* **The Dried Flower Atelier**
* **Rustic Blooms**
* **The Flower Alchemist**
* **Whispers of Lavender** (if you specialize in lavender)
* **The Dried Flower Market**

**Emphasizing Natural & Eco-Friendly:**

* **Earth & Bloom**
* **The Dried Garden**
* **Nature's Hues**
* **Botanicals & Soul**
* **The Eco Bloom**
* **Wildflower Dreams**

**Playing with a Playful Tone:**

* **Forever in Bloom**
* **The Dried Flower Bar**
* **Bloom & Beyond**
* **The Dusty Bouquet**
* **Forever Flowers**

**Things to Consider When Choosing a Name:**

* **Target audience:** Who are you trying to reach wi

✅ Recommended. The prompt below is to the point and concise.

In [7]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Flower Shop Names for Dried Bouquets:

**Short & Sweet:**

* Bloom & Dust
* The Dried Petal
* Timeless Blooms
* Everlasting Florals
* Willow & Wild
* Petal & Paper
* Thistle & Twig

**Descriptive & Elegant:**

* The Dried Garden
* The Botanical Collection
* Whispers of Bloom
* The Bloom Archive
* Sun-Kissed Flowers
* Dusty Blooms
* The Everlasting Bouquet

**Whimsical & Playful:**

* Bloom Forever
* The Dried Flower Fairy
* Petal & Plume
* Bloom & Dust Co.
* The Flower Whisperer
* The Dusty Bouquet
* The Forever Bloom

**Unique & Creative:**

* The Paper Petal
* Bloom & Bone
* The Wildflower Archive
* Sun & Stone Florals
* The Botanical Muse
* The Bloom Library
* The Flower Alchemist

**Location Specific:**

* [City Name] Dried Florals
* [Neighborhood Name] Blooms
* [Landmark] Flower Co.

**Bonus Tip:** Consider using a play on words or a pun for a more memorable name. For example:

* "Dried & True"
* "The Bloom & Bustle"
* "The Dusty Bouquet"

**Remember to check for availability o

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [8]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth, the third planet from the sun, is a vibrant and dynamic world teeming with life. Here's a glimpse into its fascinating features:

**The Basics:**

* **Size:**  A diameter of 12,742 kilometers (7,926 miles)
* **Mass:** 5.972 × 10^24 kilograms 
* **Distance from the Sun:** 149.6 million kilometers (93 million miles)
* **Orbital Period:** 365.25 days 
* **Rotation Period:** 23.93 hours
* **Natural Satellite:** The Moon

**Composition:**

* **Surface:**  70% water, 30% land
* **Atmosphere:** Primarily nitrogen and oxygen, with trace amounts of other gases
* **Core:** A solid inner core made of iron and nickel, surrounded by a liquid outer core

**Unique Features:**

* **Plate Tectonics:** The Earth's surface is divided into tectonic plates that move and interact, causing earthquakes, volcanoes, and mountain formation.
* **Magnetic Field:**  The Earth's core generates a magnetic field that shields us from harmful solar radiation.
* **Hydrosphere:**  The vas

✅ Recommended. The prompt below is specific and well-defined.

In [9]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What Makes Earth Unique:

**1. Presence of Liquid Water:** 
- Earth is the only known planet with liquid water on its surface, essential for life as we know it.
- Water acts as a solvent, facilitates chemical reactions, and regulates temperature.

**2. Active Geology:** 
- Earth's plate tectonics system constantly reshapes the surface, creating mountains, valleys, and volcanic activity.
- This activity helps regulate the climate and provides a diverse range of habitats.

**3. Atmospheric Composition:** 
- Earth's atmosphere contains a unique mix of gases, including oxygen, nitrogen, and carbon dioxide, which supports life.
- The atmosphere also protects us from harmful radiation and regulates temperature.

**4. Biosphere:** 
- Earth is the only known planet to harbor a wide variety of life forms, from microscopic bacteria to complex animals.
- This rich biodiversity creates a complex ecosystem and is essential for maintaining the planet's health.

**5. Magnetic Field:** 
- Earth's m

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [10]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's break down these two questions:

**1. Best Method of Boiling Water:**

There's no single "best" method, as it depends on your needs and resources. Here's a breakdown:

* **Stovetop:**  Most common and efficient for everyday use. 
    * **Pros:** Simple, fast, works with various cookware.
    * **Cons:** Requires an open flame or electric burner.
* **Microwave:**  Quick for small amounts of water.
    * **Pros:** Convenient, fast for small volumes.
    * **Cons:** Can only heat small quantities, not suitable for all cookware.
* **Electric Kettle:** Dedicated device for boiling water quickly and safely.
    * **Pros:** Fast, automatic shut-off, often features temperature control.
    * **Cons:** Can be expensive compared to stovetop methods.
* **Camp Stove:** Useful for outdoor or camping.
    * **Pros:** Portable, can be used with various fuels.
    * **Cons:** Requires fuel source, potentially slower than other methods.

**2. Why is the Sky Blue?**

The blue color of the sky is d

✅ Recommended. The prompts below asks one task a time.

In [11]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There's no single "best" method of boiling water, as it depends on your needs and resources. Here's a breakdown of different methods and when they might be best:

**Methods:**

* **Stovetop:**
    * **Pros:** Most common, versatile, and efficient for larger volumes. 
    * **Cons:** Requires constant attention, potential for spills or burning, not ideal for small amounts.
* **Microwave:**
    * **Pros:** Fast, convenient for small amounts, easy cleanup.
    * **Cons:** Can be uneven heating, potential for splattering, limited to small volumes.
* **Electric Kettle:**
    * **Pros:** Fast, efficient, safe, convenient for multiple uses (tea, coffee, etc.).
    * **Cons:** Requires additional appliance, not as versatile as a stovetop.
* **Camping Stove:**
    * **Pros:** Portable, ideal for outdoor use.
    * **Cons:** May be less efficient than stovetop, limited by fuel source.

**Factors to Consider:**

* **Volume of Water:** Larger volumes are best on a stovetop, while microwaves are su

In [12]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is made up of all colors of the rainbow.**  These colors have different wavelengths.
* **When sunlight enters the Earth's atmosphere, it interacts with the tiny particles of air, like nitrogen and oxygen.**  These particles are much smaller than the wavelengths of visible light.
* **Shorter wavelengths, like blue and violet, are scattered more effectively by these tiny particles than longer wavelengths, like red and orange.** This scattering is called Rayleigh scattering.
* **As a result, we see the scattered blue light coming from all directions, making the sky appear blue.**

Here are some additional points:

* **The sky appears red at sunset and sunrise because the sunlight has to travel through more of the atmosphere to reach our eyes.** This means more blue light is scattered away, leaving more red light to reach our eyes.
* **The color of the sky can also be affected by oth

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [13]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is, please check a calendar or ask a device with internet access. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [ ]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [ ]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.